<a href="https://colab.research.google.com/github/rjac-ml/Gender-Bias-Wiki/blob/add-colab-code/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!rm -r sample_data/

rm: cannot remove 'sample_data/': No such file or directory


In [56]:
from psutil import virtual_memory
import os 

ram_gb = virtual_memory().total / 1e9
NCPU = os.cpu_count()

print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print("Number  of CPU {}".format(NCPU))

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Your runtime has 54.8 gigabytes of available RAM

Number  of CPU 8
Wed May 25 12:12:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    37W / 250W |   4501MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                     

### Libraries

In [7]:
%%capture
!pip install datasets transformers

In [8]:
%%capture
!pip install -U spacy
!python -m spacy download en_core_web_lg

In [57]:
import pandas as pd
import numpy as np
from datasets import load_dataset,load_from_disk
from transformers import pipeline
import spacy
import re

In [58]:
#spacy.prefer_gpu()
nlp = spacy.load("en_core_web_lg",disable=["tagger", "parser","lemmatizer","textcat"])

### Data Sourcing

Clean articles from Wikipedia https://huggingface.co/datasets/wikipedia 

In [ ]:
df = load_dataset("wikipedia", "20220301.en")

Reusing dataset wikipedia (/root/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


  0%|          | 0/1 [00:00<?, ?it/s]

### Data Understanding

In [ ]:
df

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 6458670
    })
})

In [ ]:
df.get("train")["title"][17]

'Allan Dwan'

In [ ]:
text = df.get("train")[17].get("text")

In [ ]:
text

'Allan Dwan (born Joseph Aloysius Dwan; April 3, 1885 – December 28, 1981) was a pioneering Canadian-born American motion picture director, producer, and screenwriter.\n\nEarly life\nBorn Joseph Aloysius Dwan in Toronto, Ontario, Canada, Dwan, was the younger son of commercial traveler of woolen clothing Joseph Michael Dwan (1857–1917) and his wife Mary Jane Dwan, née Hunt. The family moved to the United States when he was seven years old on December 4, 1892 by ferry from Windsor to Detroit, according to his naturalization petition of August 1939. His elder brother, Leo Garnet Dwan (1883–1964), became a physician.\n\nAllan Dwan studied engineering at the University of Notre Dame and then worked for a lighting company in Chicago. He had a strong interest in the fledgling motion picture industry, and when Essanay Studios offered him the opportunity to become a scriptwriter, he took the job. At that time, some of the East Coast movie makers began to spend winters in California where the c

### Data Selection

In [ ]:
sdf = df.get("train").select([0,3,5])

In [ ]:
sdf = sdf.map(lambda line: {"bio_flag": len(re.findall(r"\b(biography|early life)\b",line["text"].lower()))})

  0%|          | 0/3 [00:00<?, ?ex/s]

In [ ]:
df = df.map(lambda line: {"bio_flag": len(re.findall(r"\b(biography|early life)\b",line["text"].lower()))})

  0%|          | 0/6458670 [00:00<?, ?ex/s]

In [ ]:
bios = df.get("train").filter(lambda line: line['bio_flag']>0)

  0%|          | 0/6459 [00:00<?, ?ba/s]

In [ ]:
bios.save_to_disk("/content/drive/MyDrive/pipeline/wiki")

Flattening the indices:   0%|          | 0/573 [00:00<?, ?ba/s]

In [ ]:
sdf = bios.select([0,3,5])

In [ ]:
def sentences(line):
    chunks = []
    for l in line["text"]:
        chunks += [l for l in l.split("\n\n") if len(l) > 30]
    return {'chunks': chunks}

In [ ]:
bios = bios.map(lambda line: {"intro":line["text"].split("\n\n")[0]} )

  0%|          | 0/572797 [00:00<?, ?ex/s]

In [ ]:
bios = bios.map(lambda lines:{"PERSON": [np.array([e.label_ == "PERSON" for e in nlp(i).ents]).any() for i in lines["intro"]] },batched=True,num_proc=4)

#0:   0%|          | 0/144 [00:00<?, ?ba/s]

#1:   0%|          | 0/144 [00:00<?, ?ba/s]

#2:   0%|          | 0/144 [00:00<?, ?ba/s]

#3:   0%|          | 0/144 [00:00<?, ?ba/s]

In [ ]:
bios.save_to_disk("/content/drive/MyDrive/pipeline/wiki")

In [ ]:
bios = bios.filter(lambda line : line["PERSON"] == True)

  0%|          | 0/573 [00:00<?, ?ba/s]

In [ ]:
lines = bios.map(sentences,batched=True,remove_columns=bios.column_names)

  0%|          | 0/531 [00:00<?, ?ba/s]

In [ ]:
lines

Dataset({
    features: ['chunks'],
    num_rows: 6922383
})

In [ ]:
lines.save_to_disk("/content/drive/MyDrive/pipeline/bio")

In [ ]:
lines[125127:125129]

{'chunks': ['With the financial backing of several Jewish investors, including one of the owners, Abraham Bankier, Schindler signed an informal lease agreement on the factory on 13 November 1939 and formalised the arrangement on 15 January 1940. He renamed it Deutsche Emailwarenfabrik (German Enamelware Factory) or DEF, and it soon became known by the nickname "Emalia". He initially acquired a staff of seven Jewish workers (including Abraham Bankier, who helped him manage the company) and 250 non-Jewish Poles. At its peak in 1944, the business employed around 1,750 workers, a thousand of whom were Jews. Schindler also helped run Schlomo Wiener Ltd, a wholesale outfit that sold his enamelware, and was leaseholder of Prokosziner Glashütte, a glass factory.',
  "Schindler's ties with the Abwehr and his connections in the Wehrmacht and its Armaments Inspectorate enabled him to obtain contracts to produce enamel cookware for the military. These connections also later helped him protect his 

In [ ]:
lines = lines.map(lambda line: {"F":len(re.findall(r"\b(she|her|hers|herself)\b",line.get("chunks").lower())),"M":len(re.findall(r"\b(he|his|him|himself)\b",line.get("chunks").lower()))},num_proc=4)

#0:   0%|          | 0/1730596 [00:00<?, ?ex/s]

#1:   0%|          | 0/1730596 [00:00<?, ?ex/s]

#3:   0%|          | 0/1730595 [00:00<?, ?ex/s]

#2:   0%|          | 0/1730596 [00:00<?, ?ex/s]

In [ ]:
lines[30004:30010]

{'F': [7, 4, 0, 4, 2, 3],
 'M': [1, 3, 0, 0, 0, 1],
 'chunks': ['Although Czolgosz repeatedly denied Goldman\'s involvement, the police held her in close custody, subjecting her to what she called the "third degree". She explained her housemates\' distrust of Czolgosz, and the police finally recognized that she had not had any significant contact with the attacker. No evidence was found linking Goldman to the attack, and she was released after two weeks of detention. Before McKinley died, Goldman offered to provide nursing care, referring to him as "merely a human being". Czolgosz, despite considerable evidence of mental illness, was convicted of murder and executed.',
  'Throughout her detention and after her release, Goldman steadfastly refused to condemn Czolgosz\'s actions, standing virtually alone in doing so. Friends and supporters—including Berkman—urged her to quit his cause. But Goldman defended Czolgosz as a "supersensitive being" and chastised other anarchists for abandoning

In [ ]:
lines.save_to_disk("/content/drive/MyDrive/pipeline/bio")

In [ ]:
lines = load_from_disk("/content/drive/MyDrive/pipeline/bio")

In [ ]:
lines

Dataset({
    features: ['chunks', 'F', 'M'],
    num_rows: 6922383
})

In [ ]:
lines = lines.map(lambda line : {"diff":line["F"]-line["M"]},num_proc=NCPU)

Loading cached processed dataset at /content/drive/MyDrive/pipeline/bio/cache-65a01c1d072a5433.arrow


Loading cached processed dataset at /content/drive/MyDrive/pipeline/bio/cache-46eb70deea0f6212.arrow


Loading cached processed dataset at /content/drive/MyDrive/pipeline/bio/cache-f8825f07d1d84f28.arrow


Loading cached processed dataset at /content/drive/MyDrive/pipeline/bio/cache-3c54dca19ce9840a.arrow


In [ ]:
def get_gender(lines):
    # -1: Male ,0: Neutral, 1: Female
    return {"target":np.sign(lines["diff"])}

In [ ]:
lines = lines.map(get_gender,num_proc=NCPU,batched=True)

Loading cached processed dataset at /content/drive/MyDrive/pipeline/bio/cache-10083a73ab6cde0c.arrow


Loading cached processed dataset at /content/drive/MyDrive/pipeline/bio/cache-d15fd53a053ad169.arrow


Loading cached processed dataset at /content/drive/MyDrive/pipeline/bio/cache-8a4f9ace01966eda.arrow


Loading cached processed dataset at /content/drive/MyDrive/pipeline/bio/cache-3dd58fe7aaa13a2b.arrow


In [ ]:
lines = lines.rename_column("chunks","features")
lines = lines.remove_columns(["F","M","diff"])

In [ ]:
lines = lines.train_test_split(test_size=0.3)

In [ ]:
lines.save_to_disk("/content/drive/MyDrive/pipeline/dataset")

Flattening the indices:   0%|          | 0/4846 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/2077 [00:00<?, ?ba/s]

In [ ]:
train_target_count = lines.get("train").to_pandas()["target"].value_counts()

In [ ]:
test_target_count = lines.get("test").to_pandas()["target"].value_counts()

In [ ]:
train_target_count/train_target_count.sum()

-1    0.469884
 0    0.394263
 1    0.135852
Name: target, dtype: float64

### Data Preprocessing

In [238]:
import string
punctuation = re.sub(r'[,|.|;|\-|\'|/]',"",string.punctuation)

In [211]:
!cp -r /content/drive/MyDrive/pipeline/dataset ./

In [212]:
lines = load_from_disk("./dataset")

In [213]:
lines = lines.map(lambda line: {"text": [s.translate(str.maketrans('', '',punctuation)) for s in line["features"]]},batched=True,num_proc=NCPU)

#1:   0%|          | 0/606 [00:00<?, ?ba/s]

#0:   0%|          | 0/606 [00:00<?, ?ba/s]

#3:   0%|          | 0/606 [00:00<?, ?ba/s]

#2:   0%|          | 0/606 [00:00<?, ?ba/s]

#4:   0%|          | 0/606 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/260 [00:00<?, ?ba/s]

#4:   0%|          | 0/260 [00:00<?, ?ba/s]

#5:   0%|          | 0/260 [00:00<?, ?ba/s]

#6:   0%|          | 0/260 [00:00<?, ?ba/s]

#7:   0%|          | 0/260 [00:00<?, ?ba/s]

In [190]:
#sustitude the proper noun
#sustitude pronouns 
#remove special caracters
#re string

In [214]:
lines = lines.map(
    lambda line: {"entities":[ [ent.text for ent in docs.ents if ent.label_=="PERSON"] for docs in list(nlp.pipe(line["features"]))]}
    ,batched=True
    ,batch_size=2_000
)

  0%|          | 0/2423 [00:00<?, ?ba/s]

  0%|          | 0/1039 [00:00<?, ?ba/s]

In [215]:
lines = lines.map(lambda line: {"entities": list(np.unique(line["entities"])) },num_proc=NCPU )

#0:   0%|          | 0/605709 [00:00<?, ?ex/s]

#1:   0%|          | 0/605709 [00:00<?, ?ex/s]

#2:   0%|          | 0/605709 [00:00<?, ?ex/s]

#3:   0%|          | 0/605709 [00:00<?, ?ex/s]

#5:   0%|          | 0/605708 [00:00<?, ?ex/s]

...

#3:   0%|          | 0/259589 [00:00<?, ?ex/s]

#4:   0%|          | 0/259589 [00:00<?, ?ex/s]

#5:   0%|          | 0/259589 [00:00<?, ?ex/s]

#6:   0%|          | 0/259589 [00:00<?, ?ex/s]

#7:   0%|          | 0/259589 [00:00<?, ?ex/s]

In [216]:
lines.save_to_disk("./preprocess")

In [217]:
!cp -r preprocess /content/drive/MyDrive/pipeline

In [218]:
lines.get("train")[9001]

{'entities': ['Pickett', 'Turney'],
 'features': "Turney's unit arrived in Virginia on May 8, and was eventually attached to the Army of Northern Virginia. The unit took part in the First Battle of Bull Run in July 1861, and spent the subsequent months patrolling the Potomac River region. In 1862, the unit took part in the Shenandoah Campaign, the Seven Days Battles, and the Peninsula Campaign. At the Battle of Fredericksburg in December 1862, Turney was shot in the mouth, and never returned to active fighting. While he was away recovering, his unit fought in the Battle of Gettysburg, marching with the left flank of Pickett's Charge.  Turney was given an administrative command in Florida in 1864, and remained there until the end of the war.",
 'target': -1,
 'text': "Turney's unit arrived in Virginia on May 8, and was eventually attached to the Army of Northern Virginia. The unit took part in the First Battle of Bull Run in July 1861, and spent the subsequent months patrolling the Poto

In [228]:
def preprocessing(line):
    texts = line["text"]
    entities = line["entities"]
    #entities = [[e.replace("(","\(") for e in ent] for ent in entities]
    texts = [re.sub(r'\b({})\b'.format('|'.join(e)),'a entity',t) if len(e)>0 else t for t,e in zip(texts,entities)]
    texts = [re.sub(r"\b(she|her|hers|herself|he|his|him|himself)\b","a entity",t.lower()) for t in texts]
    texts = [' '.join(t.replace("\n",' ').replace("\'s"," ").replace("\'","'").split()) for t in texts]
    return {"text":texts}

In [229]:
sample = lines.get("train").shuffle(8976).select(range(100))

Loading cached shuffled indices for dataset at dataset/train/cache-d6c454c5274a3967.arrow


In [230]:
sample = sample.map(preprocessing,batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [232]:
sample[71]

{'entities': ['Dillon Francis', 'Skyy Goldwynne'],
 'features': "Dillon Francis released a seven-episode comedy series titled Like and Subscribe in 2018, available exclusively on Funny or Die and Amazon Video. The show follows Skyy Goldwynne (Dillon Francis), Hollywood's biggest manager who locks 4 of his influencer clients in a house with a camera crew.",
 'target': -1,
 'text': 'a entity released a sevenepisode comedy series titled like and subscribe in 2018, available exclusively on funny or die and amazon video. the show follows a entity a entity, hollywood biggest manager who locks 4 of a entity influencer clients in a house with a camera crew.'}